Learning  model from hugging face:
https://huggingface.co/google/mt5-base


In [7]:
!pip install transformers sentencepiece datasets

In [17]:
from datasets import load_dataset
from google.colab import drive
from IPython.display import display
from IPython.html import widgets
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

sns.set()

In [26]:
##drive.mount('/content/gdrive')
## help(load_dataset)
## help(torch)
## help(transformers)
## help(tqdm_notebook)
## help(AdamW)
help(AutoModelForSeq2SeqLM)

Help on class AutoModelForSeq2SeqLM in module transformers.models.auto.modeling_auto:

class AutoModelForSeq2SeqLM(transformers.models.auto.auto_factory._BaseAutoModelClass)
 |  AutoModelForSeq2SeqLM(*args, **kwargs)
 |  
 |  This is a generic model class that will be instantiated as one of the model classes of the library (with a sequence-to-sequence language modeling head) when created
 |  with the [`~AutoModelForSeq2SeqLM.from_pretrained`] class method or the [`~AutoModelForSeq2SeqLM.from_config`] class
 |  method.
 |  
 |  This class cannot be instantiated directly using `__init__()` (throws an error).
 |  
 |  Method resolution order:
 |      AutoModelForSeq2SeqLM
 |      transformers.models.auto.auto_factory._BaseAutoModelClass
 |      builtins.object
 |  
 |  Class methods defined here:
 |  
 |  from_config(**kwargs) from builtins.type
 |      Instantiates one of the model classes of the library (with a sequence-to-sequence language modeling head) from a configuration.
 |      


In [10]:
# Use 'google/mt5-small' for non-pro cloab users
model_repo = 'google/mt5-base'
model_path = '/content/gdrive/My Drive/mt5_translation.pt'
max_seq_len = 20

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)
## help(tokenizer)
## type(tokenizer)

In [25]:
# Model description: https://huggingface.co/google/mt5-base
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cuda()

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [34]:
token_ids = tokenizer.encode(
    '<jp> This will be translated to Japanese! (hopefully)',
    return_tensors='pt').cuda()
help(token_ids)

model_out = model.generate(token_ids)
print(model_out)

output_text = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(model_out[0]))
print(output_text)

tensor([[  1042,   3889,    669,   1494,    898,    390,  37194,    285,    288,
          30865,    309,    274, 116024,  11994,    271,      1]],
       device='cuda:0')
tensor([[     0, 250099,    259,    264,    259, 228700,      1]],
       device='cuda:0')
<pad> <extra_id_0> - issuu</s>


In [35]:
example_input_str = '<jp> This is just a test nbuig.'
# example_input_str = 'これは普通のテスト'
input_ids = tokenizer.encode(example_input_str, return_tensors='pt')
print('Input IDs:', input_ids)

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print('Tokens:', tokens)

Input IDs: tensor([[1042, 3889,  669, 1494,  339, 1627,  259,  262, 2978,  259,  272, 1982,
         1315,  260,    1]])
Tokens: ['▁<', 'jp', '>', '▁This', '▁is', '▁just', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>']


In [36]:
sorted(tokenizer.vocab.items(), key=lambda x: x[1])

[('<pad>', 0),
 ('</s>', 1),
 ('<unk>', 2),
 ('<0x00>', 3),
 ('<0x01>', 4),
 ('<0x02>', 5),
 ('<0x03>', 6),
 ('<0x04>', 7),
 ('<0x05>', 8),
 ('<0x06>', 9),
 ('<0x07>', 10),
 ('<0x08>', 11),
 ('<0x09>', 12),
 ('<0x0A>', 13),
 ('<0x0B>', 14),
 ('<0x0C>', 15),
 ('<0x0D>', 16),
 ('<0x0E>', 17),
 ('<0x0F>', 18),
 ('<0x10>', 19),
 ('<0x11>', 20),
 ('<0x12>', 21),
 ('<0x13>', 22),
 ('<0x14>', 23),
 ('<0x15>', 24),
 ('<0x16>', 25),
 ('<0x17>', 26),
 ('<0x18>', 27),
 ('<0x19>', 28),
 ('<0x1A>', 29),
 ('<0x1B>', 30),
 ('<0x1C>', 31),
 ('<0x1D>', 32),
 ('<0x1E>', 33),
 ('<0x1F>', 34),
 ('<0x20>', 35),
 ('<0x21>', 36),
 ('<0x22>', 37),
 ('<0x23>', 38),
 ('<0x24>', 39),
 ('<0x25>', 40),
 ('<0x26>', 41),
 ('<0x27>', 42),
 ('<0x28>', 43),
 ('<0x29>', 44),
 ('<0x2A>', 45),
 ('<0x2B>', 46),
 ('<0x2C>', 47),
 ('<0x2D>', 48),
 ('<0x2E>', 49),
 ('<0x2F>', 50),
 ('<0x30>', 51),
 ('<0x31>', 52),
 ('<0x32>', 53),
 ('<0x33>', 54),
 ('<0x34>', 55),
 ('<0x35>', 56),
 ('<0x36>', 57),
 ('<0x37>', 58),
 ('<0x38>',

In [32]:
# Source: https://huggingface.co/datasets/alt
dataset = load_dataset('alt')

Generating train split:   0%|          | 0/18088 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [38]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [40]:
train_dataset[1]

{'SNT.URLID': '80188',
 'SNT.URLID.SNTID': '2',
 'url': 'http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal',
 'translation': {'bg': 'অ্যান্ড্রিয়া মেসি চার মিনিটের মাথায় ট্রাইব্রেকারের মাধ্যমে ইটালির পক্ষে একটি গোল স্কোর করেন।',
  'en': 'Andrea Masi opened the scoring in the fourth minute with a try for Italy.',
  'en_tok': 'Andrea Masi opened the scoring in the fourth minute with a try for Italy .',
  'fil': 'Si Andrea Masi ang nagsimula na makapuntos sa Italya sa ika-apat na minuto ng laro.',
  'hi': 'एंड्रिया मैसी ने चौथे मिनट में इटली के लिए ट्राई के द्वारा स्कोरिंग की शुरुआत करी।',
  'id': 'Andrea Masi membuka skor di menit keempat dengan satu try untuk Italia.',
  'ja': 'アンドレア・マージが開始4分後のトライでイタリアにとって最初の得点を入れた。',
  'khm': 'អេនត្រាម៉ាស៊ីបានស៊ុតចូល ក្នុងនាទីទីបួនគ្រដំបូងនៃពាក់កណ្តាលសំរាប់អ៊ីតាលី។',
  'lo': 'ແອນເດຣຍ ມາຊີ່ ໄດ້ເປີດການທຳຄະແນນໃນນາທີທີ່ສີ່ໃຫ້ແກ່ອິຕາລີ.',
  'ms': 'Andrea Masi telah membuka jaringan pada minit keempat dengan percubaan untuk Itali.',
  'm

In [41]:
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'ja': '<jp>',
    'zh': '<zh>'
}

In [42]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(250103, 768)

In [43]:
token_ids = tokenizer.encode(
    example_input_str, return_tensors='pt', padding='max_length',
    truncation=True, max_length=max_seq_len)
print(token_ids)

tokens = tokenizer.convert_ids_to_tokens(token_ids[0])
print(tokens)

tensor([[250102,   1494,    339,   1627,    259,    262,   2978,    259,    272,
           1982,   1315,    260,      1,      0,      0,      0,      0,      0,
              0,      0]])
['<jp>', '▁This', '▁is', '▁just', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [44]:
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

def encode_target_str(text, tokenizer, seq_len,
                      lang_token_map=LANG_TOKEN_MAPPING):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return token_ids[0]

def format_translation_data(translations, lang_token_map,
                            tokenizer, seq_len=128):
  # Choose a random 2 languages for in i/o
  langs = list(lang_token_map.keys())
  input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

  # Get the translations for the batch
  input_text = translations[input_lang]
  target_text = translations[target_lang]

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, lang_token_map)

  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, lang_token_map)

  return input_token_ids, target_token_ids

def transform_batch(batch, lang_token_map, tokenizer):
  inputs = []
  targets = []
  for translation_set in batch['translation']:
    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_seq_len)

    if formatted_data is None:
      continue

    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
  dataset = dataset.shuffle()
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, lang_token_map, tokenizer)

In [47]:
# Testing `data_transform`
in_ids, out_ids = format_translation_data(
    train_dataset[0]['translation'], LANG_TOKEN_MAPPING, tokenizer)

print(' '.join(tokenizer.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Testing data generator
data_gen = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, 8)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)

<zh> ▁ フランス の パリ 、 パル ク ・ デ ・ プラン ス で 行われた 2007 年 ラグビー ワールド カップ の プール C で 、 イタリア は 31 対 5 で ポル ト ガル を下 した 。 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
▁ 意大利 在 法国 巴黎 王子 公园 体育 场 举办 的 2007 年 橄 榄 球 世界杯 C 组 以 3 1-5 击 败 葡萄 牙 。 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

In [56]:
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [57]:
# Constants
n_epochs = 8
batch_size = 16
print_freq = 50
checkpoint_freq = 1000
lr = 5e-4
n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

In [58]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [59]:
losses = []

In [60]:
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)

In [53]:
for epoch_idx in range(n_epochs):
  # Randomize data order
  data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)

  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):
    optimizer.zero_grad()

    # Forward pass
    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)

    # Calculate loss and update weights
    loss = model_out.loss
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Print training update info
    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))

    if (batch_idx + 1) % checkpoint_freq == 0:
      test_loss = eval_model(model, test_dataset)
      print('Saving model with test loss of {:.3f}'.format(test_loss))
      torch.save(model.state_dict(), model_path)

torch.save(model.state_dict(), model_path)

<ipython-input-53-7d82fc3c8b16>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  in tqdm_notebook(enumerate(data_generator), total=n_batches):


  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 1 | Step: 50 | Avg. loss: 11.965 | lr: 0.0002777777777777778
Epoch: 1 | Step: 100 | Avg. loss: 6.810 | lr: 0.0004994418396963608
Epoch: 1 | Step: 150 | Avg. loss: 5.390 | lr: 0.0004966510381781648
Epoch: 1 | Step: 200 | Avg. loss: 4.714 | lr: 0.0004938602366599688
Epoch: 1 | Step: 250 | Avg. loss: 4.413 | lr: 0.0004910694351417727
Epoch: 1 | Step: 300 | Avg. loss: 4.141 | lr: 0.0004882786336235767
Epoch: 1 | Step: 350 | Avg. loss: 3.994 | lr: 0.0004854878321053807
Epoch: 1 | Step: 400 | Avg. loss: 3.837 | lr: 0.00048269703058718467
Epoch: 1 | Step: 450 | Avg. loss: 3.703 | lr: 0.0004799062290689886
Epoch: 1 | Step: 500 | Avg. loss: 3.512 | lr: 0.00047711542755079264
Epoch: 1 | Step: 550 | Avg. loss: 3.468 | lr: 0.00047432462603259657
Epoch: 1 | Step: 600 | Avg. loss: 3.331 | lr: 0.00047153382451440055
Epoch: 1 | Step: 650 | Avg. loss: 3.229 | lr: 0.00046874302299620453
Epoch: 1 | Step: 700 | Avg. loss: 3.203 | lr: 0.00046595222147800846
Epoch: 1 | Step: 750 | Avg. loss: 3.079 | 

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 2 | Step: 50 | Avg. loss: 2.548 | lr: 0.0004391047108729627
Epoch: 2 | Step: 100 | Avg. loss: 2.634 | lr: 0.00043631390935476675
Epoch: 2 | Step: 150 | Avg. loss: 2.577 | lr: 0.0004335231078365707
Epoch: 2 | Step: 200 | Avg. loss: 2.521 | lr: 0.0004307323063183746
Epoch: 2 | Step: 250 | Avg. loss: 2.540 | lr: 0.00042794150480017864
Epoch: 2 | Step: 300 | Avg. loss: 2.514 | lr: 0.00042515070328198257
Epoch: 2 | Step: 350 | Avg. loss: 2.521 | lr: 0.00042235990176378655
Epoch: 2 | Step: 400 | Avg. loss: 2.558 | lr: 0.00041956910024559054
Epoch: 2 | Step: 450 | Avg. loss: 2.518 | lr: 0.0004167782987273945
Epoch: 2 | Step: 500 | Avg. loss: 2.533 | lr: 0.0004139874972091985
Epoch: 2 | Step: 550 | Avg. loss: 2.563 | lr: 0.0004111966956910025
Epoch: 2 | Step: 600 | Avg. loss: 2.448 | lr: 0.0004084058941728064
Epoch: 2 | Step: 650 | Avg. loss: 2.526 | lr: 0.00040561509265461045
Epoch: 2 | Step: 700 | Avg. loss: 2.511 | lr: 0.0004028242911364144
Epoch: 2 | Step: 750 | Avg. loss: 2.454 | l

...

Epoch: 6 | Step: 50 | Avg. loss: 1.675 | lr: 0.00018659298950658628
Epoch: 6 | Step: 100 | Avg. loss: 1.656 | lr: 0.00018380218798839026
Epoch: 6 | Step: 150 | Avg. loss: 1.680 | lr: 0.00018101138647019424
Epoch: 6 | Step: 200 | Avg. loss: 1.711 | lr: 0.00017822058495199823
Epoch: 6 | Step: 250 | Avg. loss: 1.743 | lr: 0.0001754297834338022
Epoch: 6 | Step: 300 | Avg. loss: 1.707 | lr: 0.00017263898191560617
Epoch: 6 | Step: 350 | Avg. loss: 1.768 | lr: 0.00016984818039741015
Epoch: 6 | Step: 400 | Avg. loss: 1.753 | lr: 0.0001670573788792141
Epoch: 6 | Step: 450 | Avg. loss: 1.765 | lr: 0.0001642665773610181
Epoch: 6 | Step: 500 | Avg. loss: 1.749 | lr: 0.00016147577584282204
Epoch: 6 | Step: 550 | Avg. loss: 1.744 | lr: 0.00015868497432462603
Epoch: 6 | Step: 600 | Avg. loss: 1.722 | lr: 0.00015589417280643
Epoch: 6 | Step: 650 | Avg. loss: 1.721 | lr: 0.000153103371288234
Epoch: 6 | Step: 700 | Avg. loss: 1.736 | lr: 0.00015031256977003795
Epoch: 6 | Step: 750 | Avg. loss: 1.686 | l

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 7 | Step: 50 | Avg. loss: 1.520 | lr: 0.0001234650591649922
Epoch: 7 | Step: 100 | Avg. loss: 1.555 | lr: 0.00012067425764679617
Epoch: 7 | Step: 150 | Avg. loss: 1.580 | lr: 0.00011788345612860014
Epoch: 7 | Step: 200 | Avg. loss: 1.540 | lr: 0.0001150926546104041
Epoch: 7 | Step: 250 | Avg. loss: 1.574 | lr: 0.00011230185309220809
Epoch: 7 | Step: 300 | Avg. loss: 1.533 | lr: 0.00010951105157401206
Epoch: 7 | Step: 350 | Avg. loss: 1.553 | lr: 0.00010672025005581604
Epoch: 7 | Step: 400 | Avg. loss: 1.546 | lr: 0.00010392944853762
Epoch: 7 | Step: 450 | Avg. loss: 1.579 | lr: 0.00010113864701942398
Epoch: 7 | Step: 500 | Avg. loss: 1.592 | lr: 9.834784550122795e-05
Epoch: 7 | Step: 550 | Avg. loss: 1.565 | lr: 9.555704398303193e-05
Epoch: 7 | Step: 600 | Avg. loss: 1.557 | lr: 9.27662424648359e-05
Epoch: 7 | Step: 650 | Avg. loss: 1.564 | lr: 8.997544094663988e-05
Epoch: 7 | Step: 700 | Avg. loss: 1.598 | lr: 8.718463942844385e-05
Epoch: 7 | Step: 750 | Avg. loss: 1.530 | lr: 

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 8 | Step: 50 | Avg. loss: 1.412 | lr: 6.0337128823398084e-05
Epoch: 8 | Step: 100 | Avg. loss: 1.455 | lr: 5.754632730520205e-05
Epoch: 8 | Step: 150 | Avg. loss: 1.427 | lr: 5.475552578700603e-05
Epoch: 8 | Step: 200 | Avg. loss: 1.411 | lr: 5.196472426881e-05
Epoch: 8 | Step: 250 | Avg. loss: 1.424 | lr: 4.9173922750613974e-05
Epoch: 8 | Step: 300 | Avg. loss: 1.480 | lr: 4.638312123241795e-05
Epoch: 8 | Step: 350 | Avg. loss: 1.484 | lr: 4.3592319714221926e-05
Epoch: 8 | Step: 400 | Avg. loss: 1.405 | lr: 4.08015181960259e-05
Epoch: 8 | Step: 450 | Avg. loss: 1.436 | lr: 3.801071667782988e-05
Epoch: 8 | Step: 500 | Avg. loss: 1.436 | lr: 3.521991515963385e-05
Epoch: 8 | Step: 550 | Avg. loss: 1.414 | lr: 3.2429113641437824e-05
Epoch: 8 | Step: 600 | Avg. loss: 1.433 | lr: 2.9638312123241796e-05
Epoch: 8 | Step: 650 | Avg. loss: 1.474 | lr: 2.684751060504577e-05
Epoch: 8 | Step: 700 | Avg. loss: 1.465 | lr: 2.405670908684974e-05
Epoch: 8 | Step: 750 | Avg. loss: 1.436 | lr: 2.

In [61]:
## test_sentence = test_dataset[0]['translation']['en']
## test_sentence = fopen(./transcripts/kamala-harris.txt)
test_sentence = "Israel has begun what Prime Minister Benjamin Netanyahu calls the second stage of the war by expanding its military ground operations in Gaza. We spoke with Vice President Kamala Harris this past week, as the Biden administration was trying to balance Israel's need to retaliate against Hamas with the urgent need to get relief to the Palestinian people. Vice President Harris told us she is also involved in the administration's efforts on the war in Ukraine, as well as countless intractable problems, including gun violence at home. But with the Middle East on a razor's edge, we started our conversation there. The story will continue in a moment. How close is this to becoming a regional conflict that could draw in U.S. troops? We have absolutely no intention, nor do we have any plans to send combat troops into Israel or Gaza, period. Vice President Harris told us the U.S. is not telling Israel what to do, but is providing advice, equipment, and diplomatic support. A terrorist organization, Hamas, slaughtered hundreds of young people at a concert by most estimates at least 1,400 Israelis are dead. Israel without any question has a right to defend itself. That being said, it is very important that there be no conflation between Hamas and the Palestinians. The Palestinians deserve equal measures of safety and security, self-determination and dignity, and we have been very clear that the rules of war must be adhered to, and that there be humanitarian aid that flows. She told us the U.S. wants to keep the conflict from escalating, but that's proving difficult. In the last two weeks, Hezbollah in Lebanon and the Houthis in Yemen, both proxies of Iran have launched missiles, rockets, and drones against Israel. And Iranian-backed militias have fired on U.S. troops stationed in Iraq and Syria. In response, the U.S. launched airstrikes against Iranian weapons facilities in Syria. If that weren't enough of a signal to Iran and other adversaries, the Pentagon has also deployed two imposing aircraft carrier strike groups to the region. And what's the message to Iran? Don't. As President Biden said, just don't. Exactly. One word. Pretty straightforward. Since the Hamas attack on Israel, the Vice President says she has spoken with President Isaac Herzog of Israel and joined President Biden on calls with Prime Minister Benjamin Netanyahu and Palestinian Authority President Mahmoud Abbas. President Joe Biden told us in a statement, this is as high stakes and complex a situation as it gets, and Kamala is my partner in all of it. He told us Harris' advice and counsel are invaluable. When he was Vice President, Mr. Biden famously said that he wanted to be the last person in the room with President Obama. Do you have that relationship with President Biden? I do. You do. I do. And I take that responsibility quite seriously. All of them do you meet with him? Multiple times a day, quite often unless he or I are traveling. They're in total agreement the U.S. must stand with Israel and Ukraine to democracies under attack. We are as committed to Ukraine as we've always been, to authorize additional aid, to defend itself against Russia's unprovoked aggression. That is not going to waver. Because this war in the Middle East put Ukraine on a back burner. Not for us. No, it does not. Put them on a back burner at all. Vice President Harris has visited 19 countries and met with more than 100 world leaders. But lately, she has been the administration's point person on domestic priorities, traveling the country talking up the Democrats' key issues before the 2024 election. Those she hopes will fire up the base but are bound to inflame the GOP. She went to North Carolina to mark the anniversary of the Supreme Court overturning Roe V Wade. How dare they attack our fundamental rights. How dare they attack our freedom. In Virginia, it was guns. Our nation is being torn apart by gun violence. We joined the Vice President and Second Gentlemen Doug M. Hoff on Air Force 2 for a trip to Las Vegas. It was five days after the terror attack on Israel. Welcome aboard. Welcome aboard. Welcome aboard. While in the air, the Vice President joined a secure video call with the President and their national security teams to discuss measures to keep the homeland safe. Once on the ground in Las Vegas, Vice President Harris went to the college of Southern Nevada, the eighth stop on her fight for our Freedom College tour. Because you voted, Joe Biden is President of the United States and I am Vice President of the United States. Because you voted. But nationally, the Biden Harris administration is not generating the kind of enthusiasm she's seeing on her tour. A recent CBS poll found that at the beginning of President Biden's term, 70% of young people, people under 30, said he was doing a good job. Now it's less than 50%. Why is that? What's going on? If you pull how young people feel about the climate and the warming of our planet, it pulls as one of their top concerns when we talk about what we are doing with student alone debt. It pulls very high. The challenge that we have as an administration is we got to let people know who brought it to them. That's our challenge. But it is not that the work we are doing is not very, very popular with a lot of people. She blames the disconnect in part on lack of media coverage. Still, the Vice President herself is not very popular now. At 41% of adults told CBS News they approve of the job she's doing, about the same for President Biden. We talked to her the day before the carnage in Maine, but she had told us issues like mass shootings are more important than poll numbers. You have a portfolio that includes gun violence, the root cause of migration. These are some of the most intractable issues facing the country. We've done some of the most significant gun safety laws in 30 years, but we still need an assault weapons ban. It doesn't have to be this way. There wasn't assault weapons ban at one time. It expired. Let's renew it. Most Americans say that they don't think you're doing a good job on the border. You and the administration. The number of people trying to cross the U.S. southern border is at an all time high. It's no secret that we have a broken immigration system. Short term, we need a safe, orderly, and humane border policy. And long term, we need to invest in the root causes of migration. But the bottom line, Congress needs to act. Come on, participate in the solution instead of political gamesmanship. If politics is a game, Kamala Harris has proven herself to be a savvy player, forging a career that has gone from one first to another. The child of an Indian mother and a Jamaican father, she was the first woman district attorney for San Francisco. The first woman to serve as California's attorney general. The first woman of color elected senator from California. So, definitely got. And the first woman and woman of color to be elected vice president of the United States. Being in that unique position, being that first. Yeah. Is that bringing added pressure? No doubt. No doubt. You know, my mother, she would say, Kamala, you may be the first to do many things. Make sure you're not the last. And among the responsibilities that I carry and maybe impose on myself, that is one of them. So this was... She showed us around the vice president's ceremonial office. I brought in this bust of Thurgood Marshall. And I always have him over my right shoulder in the drawer here. The desk where previous vice presidents left their signatures. Al Gore, Quail, Cheney, Harry Truman. Some of these men went on to become president, but Kamala Harris told us she is focused on getting the Biden Harris ticket re-elected next year. The GOP is using her low poll numbers and president Biden's age as a battering ram. And some Democrats are growing worried. We were talking to some Democratic donors and they have told us that should something before President Biden and he is not able to run, that there would be a free for all for who would run as president. You are in the spot that that would be a natural for you to step up, but we're hearing from donors that they would not naturally fall into line. Why is that? Well, first of all, I'm not going to engage in that hypothetical because Joe Biden is very much alive and running for re-election. But you do know, I mean that is a concern and a legitimate concern, I would say. I hear from a lot of different people, a lot of different things, but let me just tell you, I'm focused on the job. A truly am. Our democracy is on the line, Bill. And I frankly, in my head, do not have time for parlor games when we have a president who is running for re-election. That's it. Joe Biden. Conventional wisdom is that most presidential elections are won or lost on the economy. And while inflation has been coming down, prices for basics like food and shelter remain staggeringly high. We came into office during the height of a pandemic, record unemployment. And because of our economic policies, we now are reducing inflation. We have created over 14 million new jobs. We've created over 800,000 new manufacturing jobs. Wages are up. And so we've seen great progress. Considering what you are laying out as your achievements, you have the current front runner for the GOP, Donald Trump, facing what 91 criminal charges. I've lost count. Yet the Biden-Harris ticket is running neck and neck with Donald Trump. Why are you not 30 points ahead? Well, I'm not a political punting. So I'm not going to speak to that, but what I will say is this. When the American people are able to take a close look at election time on their options, I think the choice is going to be clear. Bill, we're going to win. Let me just tell you that. We're going to win. And I'm not saying it's going to be easy, but we will win. You say that with such conviction. I have no doubt. But I also have no doubt. It's going to be a lot of work. And everyone's going to have to participate. This is a democracy. Democracy. She said that word often during our interview. Despite the criticism and low poll numbers, former prosecutor Kamala Harris told us she's prepared to trust the verdict of the American people. Do you have to ask yourself, why are people seeming not to hear our message? I look at it more as let's keep getting out there. And as with any election, we've got to make our case to the American people. That's part of our responsibility. And that's this process. And that's what it is. And that's a fair process."

print('Raw input text:', test_sentence)

input_ids = encode_input_str(
    text = test_sentence,
    target_lang = 'ja',
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()
## Output is truncated for performance issues
print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

Raw input text: Israel has begun what Prime Minister Benjamin Netanyahu calls the second stage of the war by expanding its military ground operations in Gaza. We spoke with Vice President Kamala Harris this past week, as the Biden administration was trying to balance Israel's need to retaliate against Hamas with the urgent need to get relief to the Palestinian people. Vice President Harris told us she is also involved in the administration's efforts on the war in Ukraine, as well as countless intractable problems, including gun violence at home. But with the Middle East on a razor's edge, we started our conversation there. The story will continue in a moment. How close is this to becoming a regional conflict that could draw in U.S. troops? We have absolutely no intention, nor do we have any plans to send combat troops into Israel or Gaza, period. Vice President Harris told us the U.S. is not telling Israel what to do, but is providing advice, equipment, and diplomatic support. A terror

In [63]:
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=3)
# print(output_tokens)
for token_set in output_tokens:
  print(tokenizer.decode(token_set, skip_special_tokens=True))

イスラエルは、ベンジャミン・ネタニヤフ首相が率いる「
イスラエルは、ベンジャミン・ネタニヤフ首相が率いるイ
イスラエルは、ベンジャミン・ネタニヤフ首相が率いる、
